In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

In [26]:
train_sero = pd.read_csv("./train_sero.csv")
company_start_year = pd.read_csv("./company_start_year.csv")
y = pd.read_csv("./y.csv")
X = pd.read_csv("./2year.csv")

In [197]:
all_data = pd.read_csv("./y.csv")

In [38]:
sero_data = pd.read_csv("./data/sero_data.csv")

In [159]:
y

,Id,year,CR,bool
0,0,2000,7.0,False
1,1,2000,7.0,False
2,2,2000,4.0,False
3,3,2000,5.0,False
4,4,2000,5.0,False
...,...,...,...,...
41624,2186,2018,5.0,False
41625,2187,2018,3.0,False
41626,2188,2018,3.0,False
41627,2189,2018,5.0,False


## 데이터 생성
- 원하는 기간만큼의 과거 데이터를 feature로 하여 데이터셋을 만드는 함수

### make_dataset
- "CR"의 값이 0인 것은 결측데이터이기 때문에 모두 제외하고 원하는 기간만큼의 과거 데이터를 feature로 하여 데이터셋을 만듦

In [595]:
def make_dataset(term):
    dataset = y[2191*(term):][:]
    start = True # 아직 x에 데이터가 쌓아지지 않았다면 0, 쌓아졌다면 1
    stop = False 
    x = None # 데이터를 쌓을 장소
    
    y["bool"]=[False]*len(y) # y bool 칼럼 False로 초기화
    
    # 데이터를 세로로 쌓는 for문
    for idx in range(len(dataset)):
        row = dataset[idx:idx+1]
        if not row["CR"].iloc[0]:
            continue
        
        # 데이터를 가로로 쌓는 for문
        for i in range(1,term+1):
            if i==1:
                a = train_sero.loc[(train_sero["Id"]==row["Id"].iloc[0])&(train_sero["year"]==row["year"].iloc[0]-i),:]
                if not a["CR"].iloc[0]:
                    stop = True
                    break
                
            else:
                b = train_sero.loc[(train_sero["Id"]==row["Id"].iloc[0])&(train_sero["year"]==row["year"].iloc[0]-i),:]
                if not b["CR"].iloc[0]:
                    stop = True
                    break
                a = a.merge(b,on="Id")
        
        # 잘못된 데이터면 쌓지않고 넘어감
        if stop:
            stop = False
            continue
        
        # x에 데이터 쌓음
        if start:
            start = False
            x = a[:]
            display(x) 
        else:
            x = pd.concat([x,a],axis=0)
        
        # y에서 종속변수로 사용된 데이터 bool 칼럼 True로 바꾸어줌
        y["bool"][2191*(term)+idx]=True
        
    return x

### 새로운 칼럼 네임을 생성(중복되지 않게)

In [613]:
def make_colname(x):
    new_col_name = []
    for idx,i in enumerate(range(0,len(x)-33,34)):
        new_col_name += list(map(lambda x:x+f"_{idx}",x[i:i+34]))
        
    return new_col_name

## 전처리

In [634]:
def mlp_data_preprocessing(X, y):
    x = X[:]
    # feature로 사용하지 않을 feature 제거
    for i in x.columns:
        if ("year" == i.split("_")[0]):
            x.drop([i], axis=1, inplace=True)
            
    # 범주형 변수 -> one-hot encoding
    # ao, ms, cs
    label_columns = ["AO","MS","CS","CR"]
    del_columns = []
    First = True
    for i in label_columns:
        for j in x.columns:
            if i == j.split("_")[0]:
                del_columns.append(j)
                if First:
                    a = pd.get_dummies(x[j])
                    First = False
                else:
                    a = pd.concat([a,pd.get_dummies(x[j])], axis=1)
    x = pd.concat([x,a], axis=1)
    print(del_columns)
    x.drop(del_columns, axis=1, inplace=True) 
    
    # 부도위험이 높은것(8,9,10)과 부도위험이 높지 않은 것으로 분류(1~7)
    y["CR"]=y["CR"].apply(lambda x: 0 if x in [1,2,3,4,5,6,7] else 1)
    y = y.loc[y["bool"]==True,"CR"]
    
    return x, y

In [414]:
X_data, y_data = mlp_data_preprocessing(X,y[2191*2:])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['AO_x', 'AO_y', 'MS_x', 'MS_y', 'CS_x', 'CS_y', 'CR_x', 'CR_y']


<ipython-input-413-a9b1754498b3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["CR"]=y["CR"].apply(lambda x: 0 if x in [1,2,3,4,5,6,7] else 1)


### 각 term별 데이터 저장

In [735]:
for term in range(0,19):
    X = make_dataset(term)
    X = X.drop(["Id"], axis=1)
    X.columns = make_colname(list(X.columns))
    X_data, y_data = mlp_data_preprocessing(X,y[2191*term:])
    
    X_data.index=range(len(X_data))
    y_data.index=range(len(y_data))
    
    total = X_data.join(y_data)
    total.to_csv(f"./final_data/{term}year.csv",index=False)
    print(f"{term}개년 데이터 완성")

,ACT_x,ACMTC_x,ACMTP_x,ACMTR_x,ACMTS_x,AO_x,CFOA_x,CSI_x,CS_x,CBIT_x,...,NE_y,S_y,SMC_y,STB_y,TFAG_y,TA_y,TAT_y,gdp_y,CR_y,year_y
0,2.87,2.200000e+09,0.0,1.829000e+09,0.0,0.0,3.987995e+09,17100356.0,1,1.968500e+09,...,233,2.620484e+09,669758000.0,0.0,99.06,4.612571e+09,0.78,651634.4,7,2000


<ipython-input-595-3f5a9a5e8e57>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["bool"][2191*(term)+idx]=True
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['AO_x_0', 'AO_y_1', 'AO_x_2', 'AO_y_3', 'AO_x_4', 'AO_y_5', 'AO_x_6', 'AO_y_7', 'AO_x_8', 'AO_y_9', 'AO_x_10', 'AO_y_11', 'MS_x_0', 'MS_y_1', 'MS_x_2', 'MS_y_3', 'MS_x_4', 'MS_y_5', 'MS_x_6', 'MS_y_7', 'MS_x_8', 'MS_y_9', 'MS_x_10', 'MS_y_11', 'CS_x_0', 'CS_y_1', 'CS_x_2', 'CS_y_3', 'CS_x_4', 'CS_y_5', 'CS_x_6', 'CS_y_7', 'CS_x_8', 'CS_y_9', 'CS_x_10', 'CS_y_11', 'CR_x_0', 'CR_y_1', 'CR_x_2', 'CR_y_3', 'CR_x_4', 'CR_y_5', 'CR_x_6', 'CR_y_7', 'CR_x_8', 'CR_y_9', 'CR_x_10', 'CR_y_11']


<ipython-input-634-e985ba2ca28f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["CR"]=y["CR"].apply(lambda x: 0 if x in [1,2,3,4,5,6,7] else 1)


12개년 데이터 완성


...

,ACT_x,ACMTC_x,ACMTP_x,ACMTR_x,ACMTS_x,AO_x,CFOA_x,CSI_x,CS_x,CBIT_x,...,NE_y,S_y,SMC_y,STB_y,TFAG_y,TA_y,TAT_y,gdp_y,CR_y,year_y
0,2.61,1.388000e+10,0.0,9.388000e+09,0.0,0.0,6.492502e+09,21874423.0,1,-3.891006e+09,...,233,2.620484e+09,669758000.0,0.0,99.06,4.612571e+09,0.78,651634.4,7,2000


<ipython-input-595-3f5a9a5e8e57>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["bool"][2191*(term)+idx]=True
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


['AO_x_0', 'AO_y_1', 'AO_x_2', 'AO_y_3', 'AO_x_4', 'AO_y_5', 'AO_x_6', 'AO_y_7', 'AO_x_8', 'AO_y_9', 'AO_x_10', 'AO_y_11', 'AO_x_12', 'AO_y_13', 'AO_x_14', 'AO_y_15', 'AO_x_16', 'AO_y_17', 'MS_x_0', 'MS_y_1', 'MS_x_2', 'MS_y_3', 'MS_x_4', 'MS_y_5', 'MS_x_6', 'MS_y_7', 'MS_x_8', 'MS_y_9', 'MS_x_10', 'MS_y_11', 'MS_x_12', 'MS_y_13', 'MS_x_14', 'MS_y_15', 'MS_x_16', 'MS_y_17', 'CS_x_0', 'CS_y_1', 'CS_x_2', 'CS_y_3', 'CS_x_4', 'CS_y_5', 'CS_x_6', 'CS_y_7', 'CS_x_8', 'CS_y_9', 'CS_x_10', 'CS_y_11', 'CS_x_12', 'CS_y_13', 'CS_x_14', 'CS_y_15', 'CS_x_16', 'CS_y_17', 'CR_x_0', 'CR_y_1', 'CR_x_2', 'CR_y_3', 'CR_x_4', 'CR_y_5', 'CR_x_6', 'CR_y_7', 'CR_x_8', 'CR_y_9', 'CR_x_10', 'CR_y_11', 'CR_x_12', 'CR_y_13', 'CR_x_14', 'CR_y_15', 'CR_x_16', 'CR_y_17']


<ipython-input-634-e985ba2ca28f>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["CR"]=y["CR"].apply(lambda x: 0 if x in [1,2,3,4,5,6,7] else 1)


18개년 데이터 완성
